# Theoretical Background

Let us start with broadly defining what is Linear Programming and its main usage:

## Linear Programming or Linear Optimization:

Method for solving a specific kind of mathematical optimization problems, namely problems with the following setting:

- Linear function Z that needs to be optimized (minimized/maximized);
- A set of linear constraints (equalities/inequalities) on the solution;
- Non-Negativity restrictions, if needed.

A standard formulation of a linear programming problem would be:

Find vector $\mathbf{x}$ that maximizes $\mathbf{c^Tx}$ subject to constraints $\mathbf{Ax} <= \mathbf{b}$ and $\mathbf{x} >= 0$.

## Optimal Transport as a linear programming problem

Let us consider the original Kantorovich problem:

$$L_C(\mathbf{a, b}) = \min_{P\in \mathbf{U(a, b)}} \langle P, C \rangle$$

$$\mathbf{U(a, b)} = \{P \in \mathbb{R}^{n*m}_{+}: P1_m = \mathbf{a} \:\: \& \:\: P^T1_m = \mathbf{b}\}$$

To present this as a linear programming problem, as we already have the objective function $Z = \langle P, C \rangle$, we need to properly encode the constraints on $P$ as a matrix-vector equality. This can be achieved by:
1) Constructing $\mathbf{p} = vec(P)$, which basically means that we transform matrix $P$ into a vector by stacking its columns one over another;
2) Constructing the matrix representing our constraint equations - it is done using the formula $A = \bigl( \begin{matrix}\mathbf{1_m}\otimes I_n\\ I_m \otimes \mathbf{1_n^T}\end{matrix} \bigr)$. $\otimes$ in this formula corresponds to Kronecker's product, in which we multiply each element of the first matrix by the second matrix. As a result, $A \in \mathbb{R}^{(n+m)*nm}$ and encodes all $n+m$ contraints on each row and column of $P$.
3) Combining the previous two steps, we get the system $A\mathbf{p}= \bigl[ \begin{matrix} \mathbf{a} \\ \mathbf{b} \end{matrix} \bigr]$. The only thing left to note is that $\mathbb{p} \in \mathbb{R}^{nm}_{+}$.

Therefore, Kantovich OT problem can be rewritten as:

$$L_C(\mathbf{a, b}) = \min_{\substack{A\mathbf{p}= \bigl[ \begin{matrix} \mathbf{a} \\ \mathbf{b} \end{matrix} \bigr] \\ p \in \mathbb{R}^{nm}_{+}}} \mathbf{c^Tp}, \:\:\:\:\: \mathbf{c} = vec(C)$$

## Dual problem

As with the original OT problem, this linear programming representation also has a different representation. To derive it (or rather, a weak duality equivalent as presented in the book), let us first construct the Lagrangian:

$$H(\mathbf{h}) = \min_{p \in \mathbb{R}^{nm}_{+}} \mathbf{c^Tp} - \mathbf{h^T}(A\mathbf{p - q}), \:\:\:\:\: \mathbf{q} = \bigl[ \begin{matrix} \mathbf{a} \\ \mathbf{b} \end{matrix} \bigr]$$

This Lagrangian, due to relaxed constraints on $\mathbf{p}$, is bound to be no bigger than $L_C(\mathbf{a, b})$
Therefore, to approach $L_C(\mathbf{a, b})$ we need to maximize $H(\mathbf{h})$:

$$ L^1_C(\mathbf{a, b}) = \max_\mathbf{h}H(\mathbf{h}) = \max_\mathbf{h}\mathbf{h^Tq} + \min_{p \in \mathbb{R}^{nm}_{+}}(\mathbf{c}-A^T\mathbf{h})^T\mathbf{p}$$

The only part left to note from the above equation is that $(\mathbf{c}-A^T\mathbf{h})$ has to be non-negative - otherwise, min can reach negative infinity and spoil our calculations. So, final representation for the dual problem is:

$$L^1_C(\mathbf{a, b}) = \max_{\substack{\mathbf{h} \in \mathbb{R}^{n+m} \\ \mathbf{\mathbf{c} \geq A^T\mathbf{h}}}}\mathbf{h^Tq}$$

Or, equivalently,

$$L^1_C(\mathbf{a, b}) = \max_{\substack{\mathbf{h} \in \mathbb{R}^{nm+} \\ \mathbf{\mathbf{c} \geq A^T\mathbf{h}}}}\bigl[ \begin{matrix} \mathbf{a} \\ \mathbf{b} \end{matrix} \bigr]^T\mathbf{h}$$

## Linear programming solution methods

**TODO**

In [ ]:
import numpy as np
from scipy.optimize import linprog

def ot_lp_solver(cost_matrix, a, b):

    cost_matrix = np.array(cost_matrix)
    a = np.array(a)
    b = np.array(b)
    c = cost_matrix.flatten()
    A_check = np.zeros((a.size + b.size, a.size * b.size))

    for i in range(a.size):
        A_check[i, i * b.size:(i + 1) * b.size] = 1
    
    for i in range(b.size):
        A_check[a.size + i, i::b.size] = 1

    expected_check = np.concatenate((a, b))

    result = linprog(c, A_eq=A_check, b_eq=expected_check)

    if result.success:
        return result.x.reshape((a.size, b.size))
    else:
        raise ValueError("Optimization failed:", result.message)

print(ot_lp_solver([[2, 3], [5, 1]], [3, 2], [1, 4]))

[[1. 2.]
 [0. 2.]]
